# PLINK2 Genome-wide QC and Ancestry PCA Pipeline

WARNING: To replace the create_lds.sh; but hasn't been validated yet.

## Purpose
Process Mayo Tapestry WES cohort for Phase B1 ancestry debiasing

## Workflow
1. Extract CDS variants per chromosome → pgen format
2. Deduplicate variants (multiallelic handling)
3. Merge chromosomes → genome-wide dataset
4. Sample/variant QC (missingness, heterozygosity, sex, relatedness)
5. LD pruning → independent SNP set for PCA
6. Ancestry PCA → genetic ancestry coordinates

In [ ]:
import subprocess
import os
from pathlib import Path

## Setup: Parameters and Directories

In [ ]:
# Working directory
work_dir = Path("/home/ext_meehl_joshua_mayo_edu/gfm-discovery/02_genomics_domain")
os.chdir(work_dir)

# Pipeline control flags
do_sex_infer = False
make_pgen = False      # Step 1: Convert VCF → pgen per chromosome
do_dedup = False       # Step 2: Remove duplicate variant IDs
do_merge = True        # Step 3: Merge chromosomes
do_qc = True           # Step 4: Sample/variant QC filters
do_missingness = True
do_het_filter = False
do_relatedness = True
do_ld_prune = True     # Step 5: LD-based SNP pruning
do_pca = True          # Step 6: Ancestry PCA
do_summary = True      # Pipeline summary output

# Chromosomes to process (exclude M - haploid, no LD structure)
chromosomes = [str(i) for i in range(1, 23)] + ['X', 'Y']

# Computational resources
n_threads = 104

# Sample QC thresholds
mind = 0.05           # Max per-sample missingness (5%)
het_sd = 3            # Heterozygosity outlier threshold (±3 SD)

# Variant QC thresholds
geno = 0.02           # Max per-variant missingness (2%)
maf = 0.01            # Minor allele frequency filter (1%)
hwe = 1e-10           # Hardy-Weinberg equilibrium p-value

# Relatedness threshold
king_cutoff = 0.0884  # Kinship coefficient (3rd degree relatives)

# LD pruning parameters
ld_window_kb = 1000   # Window size in kb
ld_step = 50          # Step size in variant count
ld_r2 = 0.2           # r² threshold

# PCA parameters
n_pcs = 20

# Directory structure
per_chr_dir = Path("data/plink/tapestry/per_chr")
genome_dir = Path("data/plink/tapestry/genome_wide")
per_chr_dir.mkdir(parents=True, exist_ok=True)
genome_dir.mkdir(parents=True, exist_ok=True)

print(f"Working directory: {work_dir}")
print(f"Per-chromosome output: {per_chr_dir}")
print(f"Genome-wide output: {genome_dir}")

## Step 0: Infer Sex from chrX (Optional)

Uses chrX heterozygosity to infer genetic sex:
- Males: high F (~1) 
- Females: low F (~0)

In [ ]:
if do_sex_infer:
    print("Step 0: Inferring sex from chrX heterozygosity...")
    
    # Rename chrX to chr25 for plink2 compatibility
    print("Renaming chrX to chr25...")
    subprocess.run([
        "bcftools", "annotate",
        "--rename-chrs", "/dev/stdin",
        "data/tapestry/vcfs/05-merged-cohort/cohort.chrX.merged.vcf.gz",
        "-Oz", "-o", f"{per_chr_dir}/chrX_temp.vcf.gz"
    ], input=b"chrX chr25", check=True)
    
    # Make pgen for chrX
    subprocess.run([
        "plink2",
        "--vcf", f"{per_chr_dir}/chrX_temp.vcf.gz",
        "--make-pgen",
        "--out", f"{per_chr_dir}/chrX_infer",
        "--threads", str(n_threads)
    ], check=True)
    
    # Calculate heterozygosity on chrX
    subprocess.run([
        "plink2",
        "--pfile", f"{per_chr_dir}/chrX_infer",
        "--chr", "25",
        "--het",
        "--out", f"{genome_dir}/chrX_het",
        "--threads", str(n_threads)
    ], check=True)
    
    print("✓ Step 0 complete: Sex info will be generated")
    print("Review chrX_het.het file and create sex_info.txt manually if needed")
else:
    print("Skipping Step 0: Sex inference")

## Step 1: Convert VCF to PGEN (Per Chromosome)

Extract CDS variants from WES VCFs using gene coordinates.

**Expected output**: ~500k-2.5M variants per autosome, 97,422 samples

In [ ]:
if make_pgen:
    print("="*50)
    print("STEP 1: Creating per-chromosome pgen files")
    print("="*50)
    
    for chrom in chromosomes:
        print(f"[Chr {chrom}] Extracting CDS variants...")
        
        # Base command
        cmd = [
            "plink2",
            "--vcf", f"data/tapestry/vcfs/05-merged-cohort/cohort.chr{chrom}.merged.vcf.gz",
            "--extract", "range", f"data/plink/ref/gtf_parsing/HS_chr{chrom}_genes_gtf.tsv",
            "--make-pgen",
            "--out", f"{per_chr_dir}/chr{chrom}_cds",
            "--threads", str(n_threads)
        ]
        
        # Special handling for chrX (requires --split-par and sex info)
        if chrom == "X" and (genome_dir / "sex_info.txt").exists():
            cmd.insert(2, "--update-sex")
            cmd.insert(3, f"{genome_dir}/sex_info.txt")
            cmd.insert(4, "--split-par")
            cmd.insert(5, "hg38")
        
        subprocess.run(cmd, check=True)
    
    print("✓ Step 1 complete: Per-chromosome pgen files created")
else:
    print("Skipping Step 1: PGEN creation")

## Step 2: Deduplicate Variants

Handle multiallelic sites:
- Create unique IDs: CHR:POS:REF:ALT format
- Allow alleles up to 237bp
- Remove duplicate positions with inconsistent genotypes

**Expected loss**: ~0.4% per chromosome

In [ ]:
if do_dedup:
    print("="*50)
    print("STEP 2: Deduplicating variants")
    print("="*50)
    
    for chrom in chromosomes:
        print(f"[Chr {chrom}] Removing duplicates and setting unique IDs...")
        subprocess.run([
            "plink2",
            "--pfile", f"{per_chr_dir}/chr{chrom}_cds",
            "--set-all-var-ids", "@:#:$r:$a",
            "--new-id-max-allele-len", "237", "missing",
            "--rm-dup", "exclude-mismatch",
            "--make-pgen",
            "--out", f"{per_chr_dir}/chr{chrom}_cds_unique",
            "--threads", str(n_threads)
        ], check=True)
    
    print("✓ Step 2 complete: Duplicates removed, unique IDs assigned")
else:
    print("Skipping Step 2: Deduplication")

## Step 3: Merge Chromosomes

Combine all chromosomes into single genome-wide dataset.

**Expected output**: ~10-12M CDS variants, 97,422 samples

In [ ]:
if do_merge:
    print("="*50)
    print("STEP 3: Merging chromosomes")
    print("="*50)
    
    # Create merge list (all chromosomes except chr1, which serves as base)
    merge_files = []
    for chrom in chromosomes:
        if chrom != "1":
            merge_files.append(f"{per_chr_dir}/chr{chrom}_cds_unique")
    
    merge_list_path = genome_dir / "merge_list.txt"
    with open(merge_list_path, 'w') as f:
        f.write('\n'.join(merge_files))
    
    print(f"Merging {len(merge_files)} chromosomes into chr1...")
    
    cmd = [
        "plink2",
        "--pfile", f"{per_chr_dir}/chr1_cds_unique",
        "--pmerge-list", str(merge_list_path), "pfile",
        "--make-pgen",
        "--out", f"{genome_dir}/genome_wide_raw",
        "--threads", str(n_threads)
    ]
    
    # Add sex info if available
    if (genome_dir / "sex_info.txt").exists():
        cmd.insert(4, "--update-sex")
        cmd.insert(5, f"{genome_dir}/sex_info.txt")
    
    subprocess.run(cmd, check=True)
    
    print("✓ Step 3 complete: Genome-wide pgen created")
else:
    print("Skipping Step 3: Merge")

## Step 4: Quality Control

Multi-stage sample and variant filtering:
- 4a. Remove high-missingness samples (>5% missing genotypes)
- 4b. Identify heterozygosity outliers (contamination/inbreeding) [DISABLED for CDS]
- 4c. Remove related individuals (kinship < 0.0884)

**Expected removals**:
- Missingness: ~100-500 samples
- Relatedness: ~500 samples
- **Final**: ~96,000 unrelated samples for PCA

In [ ]:
if do_qc:
    print("="*50)
    print("STEP 4: Quality Control")
    print("="*50)

### 4a. Sample Call Rate Filter

In [ ]:
if do_qc and do_missingness:
    print(f"[QC-1] Filtering samples with >{mind} missingness...")
    subprocess.run([
        "plink2",
        "--pfile", f"{genome_dir}/genome_wide_raw",
        "--mind", str(mind),
        "--make-pgen",
        "--out", f"{genome_dir}/genome_qc",
        "--threads", str(n_threads)
    ], check=True)
    print("✓ Missingness filter complete")
else:
    print("Skipping QC-1: Missingness filter")

### 4b. Heterozygosity Filter

**NOTE**: Disabled for CDS-only data. CDS variants are under selection and don't follow neutral expectations needed for heterozygosity QC.

In [ ]:
if do_qc and do_het_filter:
    print(f"[QC-2] Calculating inbreeding coefficient (F)...")
    subprocess.run([
        "plink2",
        "--pfile", f"{genome_dir}/genome_qc",
        "--het",
        "--out", f"{genome_dir}/het_check",
        "--threads", str(n_threads)
    ], check=True)
    
    print(f"[QC-2] Flagging heterozygosity outliers (±{het_sd} SD)...")
    print("Manual review of het_check.het recommended")
    print("✓ Heterozygosity check complete")
else:
    print("Skipping QC-2: Heterozygosity filter (disabled for CDS data)")

### 4d. Relatedness Filtering

In [ ]:
if do_qc and do_relatedness:
    print(f"[QC-4] Identifying related individuals (kinship > {king_cutoff})...")
    subprocess.run([
        "plink2",
        "--pfile", f"{genome_dir}/genome_qc",
        "--king-cutoff", str(king_cutoff),
        "--out", f"{genome_dir}/related",
        "--threads", str(n_threads)
    ], check=True)
    
    # Count unrelated samples
    unrelated_file = genome_dir / "related.king.cutoff.in.id"
    if unrelated_file.exists():
        with open(unrelated_file) as f:
            n_unrelated = sum(1 for _ in f)
        print(f"  → {n_unrelated} unrelated samples retained for PCA")
    
    print("✓ Step 4 complete: QC filtering done")
else:
    print("Skipping QC-4: Relatedness filter")

## Step 5: LD Pruning

Remove SNPs in high linkage disequilibrium to create independent variant set.

**Purpose**: Prevent over-weighting of LD blocks in PCA

**Method**: Sliding window (1 Mb), step 50 SNPs, prune pairs with r² > 0.2

**Expected output**: ~100-200k independent SNPs

In [ ]:
if do_ld_prune:
    print("="*50)
    print("STEP 5: LD-based SNP pruning")
    print("="*50)
    print(f"Parameters: window={ld_window_kb}kb, step={ld_step}, r²>{ld_r2}")
    
    subprocess.run([
        "plink2",
        "--pfile", f"{genome_dir}/genome_qc",
        "--maf", str(maf),
        "--geno", str(geno),
        "--hwe", str(hwe),
        "--indep-pairwise", str(ld_step), "5", str(ld_r2),
        "--out", f"{genome_dir}/ld_pruned",
        "--threads", str(n_threads)
    ], check=True)
    
    # Count pruned SNPs
    pruned_file = genome_dir / "ld_pruned.prune.in"
    if pruned_file.exists():
        with open(pruned_file) as f:
            n_pruned = sum(1 for _ in f)
        print(f"  → {n_pruned} independent SNPs retained for PCA")
    
    print("✓ Step 5 complete: LD pruning done")
else:
    print("Skipping Step 5: LD pruning")

## Step 6: Ancestry PCA

Compute principal components on unrelated, LD-pruned variants.

**Purpose**: Infer genetic ancestry for Phase B1 debiasing (EUR/AFR/EAS/SAS)

**Outputs**:
- `ancestry_pca.eigenvec`: PC1-PC20 coordinates per sample
- `ancestry_pca.eigenval`: Variance explained per PC
- `ancestry_pca.acount`: Allele counts

**Next steps**: Plot PC1 vs PC2 to identify ancestry clusters → assign labels

In [ ]:
if do_pca:
    print("="*50)
    print("STEP 6: Ancestry PCA")
    print("="*50)
    print(f"Computing {n_pcs} PCs on unrelated, LD-pruned variants...")
    
    subprocess.run([
        "plink2",
        "--pfile", f"{genome_dir}/genome_qc",
        "--keep", f"{genome_dir}/related.king.cutoff.in.id",
        "--extract", f"{genome_dir}/ld_pruned.prune.in",
        "--freq", "counts",
        "--pca", str(n_pcs), "approx",
        "--out", f"{genome_dir}/ancestry_pca",
        "--threads", str(n_threads)
    ], check=True)
    
    print("✓ Step 6 complete: PCA results written")
    print("")
    print("Next steps:")
    print("  1. Plot PCs: python scripts/plot_ancestry_pca.py")
    print("  2. Assign ancestry labels via clustering or gnomAD projection")
    print("  3. Proceed to Phase B1 subspace removal debiasing")
else:
    print("Skipping Step 6: PCA")

## Pipeline Summary

In [ ]:
if do_summary:
    print("="*50)
    print("PIPELINE COMPLETE")
    print("="*50)
    print("Key outputs:")
    print(f"  - QC'd genome-wide pgen: {genome_dir}/genome_qc.{{pgen,pvar,psam}}")
    print(f"  - Unrelated sample list: {genome_dir}/related.king.cutoff.in.id")
    print(f"  - LD-pruned SNP list: {genome_dir}/ld_pruned.prune.in")
    print(f"  - Ancestry PCs: {genome_dir}/ancestry_pca.eigenvec")
    print("")
    print("For Phase B1 Week 1-2 deliverables:")
    print("  ✓ Genome-wide LD-pruned dataset ready")
    print("  ✓ Genetic ancestry PCA computed")
    print("  → Next: Ancestry stratification + subspace removal")
else:
    print("Summary disabled")